In [ ]:
from collections import defaultdict

In [ ]:
class NFA:
    def _init_(self, start, accept):
        self.start = start
        self.accept = accept

class State:
    _counter = 0  # Counter to assign unique IDs to states.
    
    def _init_(self, is_accept=False):
        self.id = f'q{State._counter}'
        State._counter += 1
        self.is_accept = is_accept
        self.transitions = defaultdict(list)  # Dictionary to store transitions.

    def _str_(self):
        return self.id

# Convert a regex to NFA.
def regex_to_nfa(regex):
    stack = []  # Stack to store NFAs created during the conversion.
    
    for char in regex:
        # If char is a regular symbol.
        if char not in ('U', '*', '(', ')'):
            start = State()
            accept = State(True)
            start.transitions[char].append(accept)
            stack.append(NFA(start, accept))
        
        # Kleene star operation.
        elif char == '*':
            old_nfa = stack.pop()
            start = State()
            accept = State(True)
            start.transitions[''].append(old_nfa.start)
            start.transitions[''].append(accept)
            old_nfa.accept.transitions[''].append(old_nfa.start)
            old_nfa.accept.transitions[''].append(accept)
            stack.append(NFA(start, accept))
        
        # Union operation.
        elif char == 'U':
            right_nfa = stack.pop()
            left_nfa = stack.pop()
            start = State()
            accept = State(True)
            start.transitions[''].append(left_nfa.start)
            start.transitions[''].append(right_nfa.start)
            left_nfa.accept.transitions[''].append(accept)
            right_nfa.accept.transitions[''].append(accept)
            stack.append(NFA(start, accept))
        
        # Concatenation operation.
        elif char == ')':
            right_nfa = stack.pop()
            left_nfa = stack.pop()
            left_nfa.accept.transitions[''].extend(right_nfa.start.transitions.values())
            stack.append(NFA(left_nfa.start, right_nfa.accept))
    
    final_nfa = stack[0]
    
    # Convert NFA states to a transition dictionary.
    transitions = {}
    visited = set()
    to_visit = [final_nfa.start]
    while to_visit:
        current = to_visit.pop()
        if current not in visited:
            visited.add(current)
            for symbol, next_states in current.transitions.items():
                transitions[(str(current), symbol)] = [str(s) for s in next_states]
                to_visit.extend(next_states)
    return transitions, str(final_nfa.start), [str(final_nfa.accept)]